# Stochastic Analysis of a simple recombinase-based oscillator

In this Jupyterbook, we implement the Gilleaspy algorithm of a recombinase-based oscillator at a single copy 

In [31]:
# Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
  
    from google.colab import drive
    drive.mount('/content/drive')
    dir = "/content/drive/My Drive/Research/2022/RecombinaseOscillator/Self_Inhibiting/"
else:
    dir = "/Users/christian/My Drive/Research/2022/RecombinaseOscillator/Self_Inhibiting/"
# ------

# This check the number of processors in the computer
#% cat /proc/cpuinfo | grep processor | wc -l

# % pip install biocircuits bokeh-catplot watermark blackcellmagic
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
import numba
import math

from scipy.signal import argrelextrema
import scipy.fftpack
import os.path

import biocircuits

# Plotting modules
import bokeh.io
import bokeh.plotting
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.io import export_svgs

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
def JoinMatrices(A,B):
    n1, m1 = A.shape
    n2, m2 = B.shape
    ZA = np.zeros((n1,m2))
    ZB = np.zeros((n2,m1))
    tmp1 = np.hstack((A,ZA))
    tmp2 = np.hstack((ZB,B)) 
    return np.vstack((tmp1,tmp2))

In [3]:
# Stoichiometry  matrix of a single switch
# R00, R10, R01, R11
S1 = np.array(
    [
     [-1, 1, 0, 0, 0],  # R00 -> R10
     [ 1,-1, 0, 0, 0], # R10 -> R00 
     [ 0,-1, 0, 1, 0], # R10 -> R11
     [ 0, 1, 0,-1, 0], # R11 -> R10

     [-1, 0, 1, 0, 0],  # R00 -> R01
     [ 1, 0,-1, 0, 0], # R01 -> R00 
     [ 0, 0,-1, 1, 0], # R01 -> R11
     [ 0, 0, 1,-1, 0], # R11 -> R01 

     [ 0, 0, 0,-1, 1], # R11 -> L00
    ],
    dtype=int,
)
#print(S1)

In [4]:
# Stoichiometry  matrix of transcription and translation
# m1, x1, y1, m2, x2, y2, c
S2 = np.array(
    [
     [ 1, 0, 0, 0], # m1 production (TX)
     [-1, 0, 0, 0], # m1 decay
     [ 0, 1, 0, 0], # x1 production (TL)
     [ 0,-1, 0, 0], # x1 decay
     [ 0,-2, 1, 0], # y1 production (dimerization)
     [ 0, 2,-1, 0], # y1 dissociation
     [ 0, 0,-1, 0], # y1 decay
     [ 0, 0, 0, 1], # z1 prod (dummy specie)
     [ 0, 0, 0,-1], # z1 decay (dummy specie)
    ],
    dtype=int,
)
# Stoichiometry  matrix of two switches
simple_update = JoinMatrices(S1,S2)
# Updating the consumption of recombinase in the switches
S0 = np.array(
    [
     [-1, 1, -1, 1,-1, 1, -1, 1, 2],  # R00 -> R10
    ],
    dtype=int,
)
simple_update[0:9,7] = S0

print(simple_update)
#simple_update.shape

[[-1.  1.  0.  0.  0.  0.  0. -1.  0.]
 [ 1. -1.  0.  0.  0.  0.  0.  1.  0.]
 [ 0. -1.  0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0. -1.  0.  0.  0.  1.  0.]
 [-1.  0.  1.  0.  0.  0.  0. -1.  0.]
 [ 1.  0. -1.  0.  0.  0.  0.  1.  0.]
 [ 0.  0. -1.  1.  0.  0.  0. -1.  0.]
 [ 0.  0.  1. -1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0. -1.  1.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -2.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  2. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.]]


In [5]:
def simple_propensity(propensities, x, t, th, rh, ph, d, g, gd, a0, d0, r, Omega, n):
    #x, y, c = population
    # Switch 1 (Promoter pointing to the right R)
    propensities[0] = a0 * x[0] * x[7]  / Omega # R00 -> R10
    propensities[1] = d0 * x[1] # R10 -> R00
    propensities[2] = a0 * x[1] * x[7]  / Omega # R10 -> R11
    propensities[3] = d0 * x[3] # R11 -> R10
    propensities[4] = a0 * x[0] * x[7]  / Omega # R00 -> R01
    propensities[5] = d0 * x[2] # R01 -> R00
    propensities[6] = a0 * x[2] * x[7]  / Omega # R01 -> R11
    propensities[7] = d0 * x[3] # R11 -> R01
    propensities[8] = r * x[3] # R11 -> L00

    # TX-TL of recombinase 1
    propensities[9] = th * x[0] # R00 -> R00 + M1
    propensities[10] = ph * x[5] # M1 -> 0
    propensities[11] = rh * x[5] # M1 -> M1 + X1
    propensities[12] = d * x[6] # X1 -> 0
    propensities[13] = g * x[6] * (x[6]-1)  / 2 / Omega # X1 + X1 -> Y1
    propensities[14] = gd * x[7] # Y1 -> X1 + X1
    propensities[15] = d * x[7] # Y1 -> 0
    
    # TX of dummy specie
    propensities[16] = 5 * x[4] # M1 -> M1 + X1
    propensities[17] = 1 * x[8] # X1 -> 0
                  

In [12]:
# Specify parameters for calculation
n = 1 # copy number of promoter
th = .3*60/n # 1/h
rh = 80 # 1/h
ph = 0.6931/10*60 # 1/h log(2)=0.6931
d = 0.6931/30*60 # 1/h log(2)=0.6931
g = 2.1*36 # 1/uM/h 
gd = g*0.01 # 1/h
r = 5 # 1/h Switching -- rate 5 and 1 for paper
a0 = 2.1*36
d0 = 0.1*a0

Omega = 600 # 600

args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 10, 51)
population_0 = np.zeros(9, dtype=int)
population_0[0] = n
population_0[7] = 0

samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=125, args=args, n_threads=4,progress_bar=True)

100%|██████████| 125/125 [00:37<00:00,  3.37it/s]


In [36]:
# Set up plots
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [150, 115] # 300 by 200
x_range = (0, time_points[-1])
y_range = (0, 400)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"


# Plot trajectories of recombinase 1 and 2
#for i in range(samples.shape[0]):
for i in range(21):
    y1 = samples[i,:,7] # samples[:,:,10][0]
    plots[0].step(time_points, y1, line_width=2, alpha= 0.5, line_join="bevel",color="grey")
    s1 = samples[i,:,4] # samples[:,:,10][0]
    plots[1].step(time_points, s1, line_width=2, alpha= 0.5, line_join="bevel",color="grey")

plots[0].step(time_points, y1, line_width=2, alpha= 1, line_join="bevel",color="black")
plots[1].step(time_points, s1, line_width=2, alpha= 1, line_join="bevel",color="black")

idx = 7 # Concentration of recombinase
max_id = samples[i,:,idx].argmax()

Tf = time_points[max_id]
Af = samples[i,max_id,idx]

plots[0].circle(Tf,Af, line_width=4, color="orange")
plots[0].circle(Tf,Af, line_width=1, color="white")

plots[1].circle(Tf,1, line_width=4, color="orange")
plots[1].circle(Tf,1, line_width=1, color="white")

plots[0].output_backend = "svg"
export_svgs(plots[0],filename="fig/Sim_Pulse_Recombinase.svg")

plots[1].output_backend = "svg"
export_svgs(plots[1],filename="fig/Sim_Pulse_switch.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [9]:
# This function finds the time of the peak
def Find_A_t(t,sample, idx):
    Av = []
    tv = []
    for i in range(sample.shape[0]):
        #max_id = np.where(sample[i,:,idx1]==1)[0][0]
        max_id = samples[i,:,idx].argmax()
        tv = np.append(tv,t[max_id])
        Av = np.append(Av,sample[i,max_id,idx])
        
    return (tv,Av)

In [40]:
# Set up plots for publication
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [150, 115] # tovisualize data (300,200)
x_range = (0, time_points[-1])
y_range = (0, 400)

plots = []
for i in range(2):
    plots.append(bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          #x_range=x_range, y_range=y_range
                          ),)
    plots[i].axis.major_label_text_font_size = "10pt"


    
idx = 7 # Concentration of recombinase
Tf, Af = Find_A_t(time_points,samples, idx)

bin0 = np.linspace(0,10,21)
hist, bin_edges = np.histogram(Tf, bin0, density=False)
#hist1 = hist/hist.sum()
plots[0].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="orange", line_color="orange", fill_alpha=0.2,line_alpha=0.)
plots[0].step(bin_edges[:-1],hist, line_width=2, mode="after", color = "orange")


bin0 = np.linspace(0,200,25)
hist, bin_edges = np.histogram(Af, bin0, density=False)
#hist2 = hist/hist.sum()
plots[1].quad(top=hist, bottom=0, left=bin_edges[:-1], right=bin_edges[1:],fill_color="orange", line_color="orange", fill_alpha=0.2,line_alpha=0.)
plots[1].step(bin_edges[:-1],hist, line_width=2, mode="after", color = "orange")

plots[0].output_backend = "svg"
export_svgs(plots[0],filename="fig/Sim_Hist_Ti.svg")

plots[1].output_backend = "svg"
export_svgs(plots[1],filename="fig/Sim_Hist_Xi.svg")

bokeh.io.show(bokeh.layouts.row(plots))

In [43]:
def contourf(x, y, z, H, palette="Spectral11"): #Spectral11  Viridis256
    """Make a filled contour plot given x, y, z data given in 2D arrays."""
    p = bokeh.plotting.figure(
         plot_width=3*48, plot_height=2*55,
        x_range=(x.min(), x.max()), y_range=(y.min(), y.max()),
        #x_axis_label="x1", y_axis_label="x2",
    )
    
    N = x.shape[1]
    z0 = np.c_[z, np.zeros(N)]
    z0[-1,-1] = H
    
    p.image(
        image=[z0], x=x.min(), y=y.min(),
        dw=(x.max() - x.min())*(1 + 1/N),
        dh=x.max() - x.min(),
        palette=palette,
        alpha=0.8,
    )

    color = LinearColorMapper(palette = palette, low = z0.min(), high = z0.max())
    cb = ColorBar(color_mapper = color, location = (0,0), width=5)
    
    #p.add_layout(cb, 'right')
    return p


In the next code, we compute the distributions for different parameters and compare them in a heat plot. We start with the translation rate $\rho$

In [84]:
args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 10, 51)
population_0 = np.zeros(9, dtype=int)
population_0[0] = n

size = 125 #125
param = (1)
scale = (1./4, 1./3, 1./2, 1., 2.,3., 4.)
idx = 7 # Concentration of recombinase

bin1 = np.linspace(0,10,21)
out1 = np.zeros(bin1.shape[0]-1)

bin2 = np.linspace(0,200,25)
out2 = np.zeros(bin2.shape[0]-1)

args_i = np.array((th, rh, ph, d, g, gd, a0, d0, r, Omega, n))
for fac in scale:
    # Update model para
    args_i[param] = args[param]*fac
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=size, args=tuple(args_i), n_threads=4,progress_bar=True)
               
    Tf, Af = Find_A_t(time_points,samples, idx)
    hist, bin_edges = np.histogram(Tf, bin1, density=False)
    out1 = np.vstack((out1,hist))
        
    hist, bin_edges = np.histogram(Af, bin2, density=False)
    out2 = np.vstack((out2,hist))
    

100%|██████████| 125/125 [00:23<00:00,  5.29it/s]


In [85]:
# Set up plots for publication
# For two digits 260 by 115 larger and 144 by 115 for half size
# For three digits 266 by 115 larger and 150 by 115 for half size
fig_size = [190, 115] # for paper size (144, 110) # for larger size (144,144)
y_range = (0, 7)
#palette = "Spectral11"
palette = "Plasma256"

x_range = (0, bin1[-1])
plots[0] = bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          )
plots[0].axis.major_label_text_font_size = "10pt"

x_range = (0, bin2[-1])
plots[1] = bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          )
plots[1].axis.major_label_text_font_size = "10pt"

# must give a vector of image data for image parameter
# Ploting distributions of peak time
plots[0].image(image=[out1[1:,:]], x=0, y=0, dw=bin1[-1], dh=7, palette=palette)
color = LinearColorMapper(palette = palette, low = out1.min(), high = out1.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
plots[0].add_layout(cb, 'right')
#plots[0].yaxis.ticker = (0.5,1.5,2.5, 3.5, 4.5, 5.5, 6.5)
plots[0].yaxis.ticker = list(np.linspace(0.5,6.5,7))

# Ploting distributions of peak/s amplitude
plots[1].image(image=[out2[1:,:]], x=0, y=0, dw=bin2[-1], dh=7, palette=palette)
color = LinearColorMapper(palette = palette, low = out2.min(), high = out2.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
plots[1].add_layout(cb, 'right')
plots[1].yaxis.ticker = list(np.linspace(0.5,6.5,7))

bokeh.io.show(bokeh.layouts.row(plots))

In the next code, we compute the distributions for different parameters and compare them in a heat plot. Here, we vary the switching rate $r$

In [88]:
args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 10, 51)
population_0 = np.zeros(9, dtype=int)
population_0[0] = n

size = 125 #125
param = (8)
scale = (1./4, 1./3, 1./2, 1., 2.,3., 4.)
idx = 7 # Concentration of recombinase

bin3 = np.linspace(0,10,21)
out3 = np.zeros(bin3.shape[0]-1)

bin4 = np.linspace(0,200,25)
out4 = np.zeros(bin4.shape[0]-1)

args_i = np.array((th, rh, ph, d, g, gd, a0, d0, r, Omega, n))
for fac in scale:
    # Update model para
    args_i[param] = args[param]*fac
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_0,time_points, size=size, args=tuple(args_i), n_threads=4,progress_bar=True)
               
    Tf, Af = Find_A_t(time_points,samples, idx)
    hist, bin_edges = np.histogram(Tf, bin3, density=False)
    out3 = np.vstack((out3,hist))
        
    hist, bin_edges = np.histogram(Af, bin4, density=False)
    out4 = np.vstack((out4,hist))

100%|██████████| 125/125 [00:30<00:00,  4.08it/s]

100%|██████████| 125/125 [00:24<00:00,  5.17it/s]

100%|██████████| 125/125 [00:19<00:00,  6.29it/s]

100%|██████████| 125/125 [00:25<00:00,  4.83it/s]

100%|██████████| 125/125 [00:25<00:00,  4.88it/s]


In [89]:
fig_size = [190, 115] # for paper size (144, 110) # for larger size (144,144)
y_range = (0, 7)
palette = "Plasma256"

x_range = (0, bin3[-1])
plots[0] = bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          )
plots[0].axis.major_label_text_font_size = "10pt"

x_range = (0, bin4[-1])
plots[1] = bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          )
plots[1].axis.major_label_text_font_size = "10pt"

# must give a vector of image data for image parameter
plots[0].image(image=[out3[1:,:]], x=0, y=0, dw=bin3[-1], dh=7, palette=palette)
color = LinearColorMapper(palette = palette, low = out3.min(), high = out3.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
plots[0].add_layout(cb, 'right')
plots[0].yaxis.ticker = list(np.linspace(0.5,6.5,7))

plots[1].image(image=[out4[1:,:]], x=0, y=0, dw=bin4[-1], dh=7, palette=palette)
color = LinearColorMapper(palette = palette, low = out4.min(), high = out4.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
plots[1].add_layout(cb, 'right')
plots[1].yaxis.ticker = list(np.linspace(0.5,6.5,7))

bokeh.io.show(bokeh.layouts.row(plots))

In the next code, we compute the distributions for different parameters and compare them in a heat plot. Here, we vary the initial conditions

In [93]:
args = (th, rh, ph, d, g, gd, a0, d0, r, Omega, n)
time_points = np.linspace(0, 10, 51)

size = 125 #125
par = 7 # corresponds to the state 7 (recombinase)
#scale = (0,3,6,9,12,15,18)
scale = (0,5,10,15,20,25,30)
idx = 7 # Concentration of recombinase

bin5 = np.linspace(0,10,21)
out5 = np.zeros(bin5.shape[0]-1)

bin6 = np.linspace(0,200,25)
out6 = np.zeros(bin6.shape[0]-1)

population_i = np.zeros(9, dtype=int)
population_i[0] = n

for x0 in scale:
    # Update model para
    population_i[par] = x0
    samples = biocircuits.gillespie_ssa(simple_propensity, simple_update, population_i,time_points, size=size, args=args, n_threads=4,progress_bar=True)
               
    Tf, Af = Find_A_t(time_points,samples, idx)
    hist, bin_edges = np.histogram(Tf, bin5, density=False)
    out5 = np.vstack((out5,hist))
        
    hist, bin_edges = np.histogram(Af, bin6, density=False)
    out6 = np.vstack((out6,hist))

100%|██████████| 125/125 [00:31<00:00,  3.91it/s]


In [100]:
fig_size = [190, 115] # for paper size (144, 110) # for larger size (144,144)
y_range = (0, 7)

palette = "Plasma256"

x_range = (0, bin5[-1])
plots[0] = bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          )
plots[0].axis.major_label_text_font_size = "10pt"

x_range = (0, bin6[-1])
plots[1] = bokeh.plotting.figure(plot_width=fig_size[0], plot_height=fig_size[1],
                          x_range=x_range,y_range=y_range
                          )
plots[1].axis.major_label_text_font_size = "10pt"

# must give a vector of image data for image parameter
plots[0].image(image=[out5[1:,:]], x=0, y=0, dw=bin5[-1], dh=7, palette=palette)
color = LinearColorMapper(palette = palette, low = out5.min(), high = out5.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
plots[0].add_layout(cb, 'right')
plots[0].yaxis.ticker = list(np.linspace(0.5,6.5,7))

plots[1].image(image=[out6[1:,:]], x=0, y=0, dw=bin6[-1], dh=7, palette=palette)
color = LinearColorMapper(palette = palette, low = out6.min(), high = out6.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
plots[1].add_layout(cb, 'right')
plots[1].yaxis.ticker = list(np.linspace(0.5,6.5,7))

bokeh.io.show(bokeh.layouts.row(plots))

In [ ]:
population_0i

array([1, 0, 0, 0, 0, 0, 0, 0, 0])